In [4]:
pip install python-dotenv openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.10.0
    Uninstalling typing_extensions-4.10.0:
      Successfully uninstalled typing_extensions-4.10.0

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [136]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv("hse.env")

True

In [137]:
import json
import os
import requests
from openai import OpenAI
import xml.etree.ElementTree as ET
from datetime import datetime

# Настройка клиента для OpenRouter
client = OpenAI(
    base_url= os.environ.get("OPENAI_BASE_URL"),  #"https://openrouter.ai/api/v1",
    api_key=os.environ.get("OPENROUTER_API_KEY"),
    default_headers={
        "HTTP-Referer": "https://hse.ru",
        "X-Title": "HSE Weather and currency App",
    },
)

model = "mistralai/mistral-7b-instruct:free"

In [138]:
def get_current_weather(address):
    """Получить погоду по адресу"""
    geo_url = "http://api.openweathermap.org/geo/1.0/direct"
    geo_params = {
        'q': f"{address},RU",
        'limit': 1,
        'appid': os.environ['WEATHERMAP_API_KEY']
    }
    geo_response = requests.get(geo_url, params=geo_params).json()
    
    if not geo_response:
        return json.dumps({"error": "Адрес не найден"})
    
    lat, lon = geo_response[0]['lat'], geo_response[0]['lon']
    
    weather_url = "https://api.openweathermap.org/data/2.5/weather"
    weather_params = {
        'lat': lat,
        'lon': lon,
        'appid': os.environ['WEATHERMAP_API_KEY'],
        'units': 'metric'
    }
    weather_response = requests.get(weather_url, params=weather_params).json()
    
    result = {
        "address": address,
        "temperature": weather_response['main']['temp'],
        "feels_like": weather_response['main']['feels_like'],
        "humidity": weather_response['main']['humidity']
    }
    return json.dumps(result)
    
get_current_weather("Saint Petersburg")

'{"address": "Saint Petersburg", "temperature": 4.44, "feels_like": 0.64, "humidity": 88}'

In [141]:


def get_currency_rate(currency_code):
    """Получить курс валюты от Центробанка России"""
    currency_codes = {
        'USD': 'R01235',
        'EUR': 'R01239',
        'CNY': 'R01375',
        'JPY': 'R01820'
    }
    
    try:
        response = requests.get('https://www.cbr.ru/scripts/XML_daily.asp')
        root = ET.fromstring(response.content)
        
        date_str = root.attrib['Date']
        current_date = datetime.strptime(date_str, "%d.%m.%Y").strftime("%Y-%m-%d")
        
        for valute in root.findall('Valute'):
            if valute.find('CharCode').text == currency_code:
                nominal = int(valute.find('Nominal').text)
                value = float(valute.find('Value').text.replace(',', '.'))
                name = valute.find('Name').text
                
                return json.dumps({
                    "date": current_date,
                    "currency": currency_code,
                    "name": name,
                    "nominal": nominal,
                    "value": round(value / nominal, 4),
                    "rate": round(value, 4)
                })
                
        return json.dumps({"error": "Валюта не найдена"})
    
    except Exception as e:
        return json.dumps({"error": str(e)})

def run_conversation(content):
    messages = [{"role": "user", "content": content}]
    
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Получить текущую погоду по адресу",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "address": {
                            "type": "string",
                            "description": "FIND ONLY the CITY or nearest TOWN in user request, translate it on English, example: Moscow",
                        }
                    },
                    "required": ["address"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_currency_rate",
                "description": "Получить курс валюты от ЦБ РФ",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "currency_code": {
                            "type": "string",
                            "enum": ["USD", "EUR", "CNY", "JPY"],
                            "description": "Код валюты: USD, EUR, CNY или JPY",
                        }
                    },
                    "required": ["currency_code"]
                }
            }
        }
    ]
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
        messages.append(response_message)
        
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)
            
            if function_name == "get_current_weather":
                print(f"Запрос погоды для: {args['address']}")
                function_response = get_current_weather(args['address'])
                
            elif function_name == "get_currency_rate":
                print(f"Запрос курса валюты: {args['currency_code']}")
                function_response = get_currency_rate(args['currency_code'])
                
            else:
                function_response = json.dumps({"error": "Функция не найдена"})
            
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            })

        final_response = client.chat.completions.create(
            model=model,
            messages=messages,
            stream=True
        )
    else:
        messages.append(response_message)
        final_response = client.chat.completions.create(
            model=model,
            messages=messages,
            stream=True
        )
    
    return final_response

if __name__ == "__main__":
    questions = [
        "Какая погода сейчас около Вышки на Покровском Бульваре в Москве?",
        "Какой текущий курс Юаня?",
        "Сколько стоит евро по курсу ЦБ?",
        "Какая ключевая ставка ЦБ будет в 2026 году?"
    ]
    
    for question in questions:
        print(f"\nВопрос: {question}")
        response = run_conversation(question)
        
        if response:
            print("Ответ:")
            for chunk in response:
                content = chunk.choices[0].delta.content or ""
                print(content, end='', flush=True)
            print()


Вопрос: Какая погода сейчас около Вышки на Покровском Бульваре в Москве?
Запрос погоды для: Moscow
Ответ:
На текущий момент в Москве, рядом с Вышкой на Покровском Бульваре, температура составляет около 17.08°C, а ощущаемая температура — 16.04°C. Влажность составляет 46%.

Вопрос: Какой текущий курс Юаня?
Запрос курса валюты: CNY
Ответ:
На дату 2025-04-22 курс юаня (CNY) составляет 11.0648 российских рублей. Пожалуйста, убедитесь, что вы проверяете актуальный курс на нужную вам дату, так как он может изменяться со временем.

Вопрос: Сколько стоит евро по курсу ЦБ?
Запрос курса валюты: EUR
Ответ:
По состоянию на 22 апреля 2025 года, курс евро по Центральному Банку составляет 93.6714 рубля.

Вопрос: Какая ключевая ставка ЦБ будет в 2026 году?
Ответ:
К сожалению, я не могу предсказать ключевую ставку ЦБ России на 2026 год, так как это требует анализа будущих экономических условий и решений регулятора. Однако я могу предоставить вам информацию о том, как ключевая ставка обычно формируется 